# Integration
This are the initial tests towards integration. One of the main analysis the time consume to run the codes 

In [18]:
from dtwinpylib.dtwinpy.Digital_Twin import Digital_Twin
from dtwinpylib.dtwinpy.interfaceDB import Database

import importlib
import sqlite3
import dtwinpylib
importlib.reload(dtwinpylib.dtwinpy.Digital_Twin) 
importlib.reload(dtwinpylib.dtwinpy.interfaceDB) 

<module 'dtwinpylib.dtwinpy.interfaceDB' from 'c:\\Users\\pedro\\Github Repositories\\Digital_Twin\\Digital Twin\\dtwinpylib\\dtwinpy\\interfaceDB.py'>

## When to Synchronize?

The problem with Sync is that we need to update the timestamp from real log to match (with a certain error) the timestamp with the digital log. Ideally the digital log starts with timestamp 0, but just in the case that some event happen rigth in the beginning of the simulation. Thus, in the case of a simulation where all the machines are busy or the free machines doesn't have any part in the queue, the digital log is just going to be update after the first busy machine finish a part, so where is the time zero for the real log in that case? 

So would be ideal to just synchronize with a trace 'Started'. Thus, for that we need to have a certain frequency to Synch, but also the initial trace need to be 'Started', so the final frequency could be a little be early or a little bit late, depending on each event 'Started' is more close to the ideal time.

Let's take this example of real log:
dT = 1/f
Tsync = To + dT

f = (1/10)
dT = 10
To = 0
Tsync = 10

Ideally, the cut in the database to Synchronize the systems would be Tsync = 10. But this is a finished trace, so we look for the most close 'started' trace. In this case, a little bit in the past in T= 00

(00, m3, s)
(10, m1, f)
(20, m2, s)
(20, m1, s)
...



In [21]:
mydt = Digital_Twin(name= "5s_determ")
mydt.run_digital_model(plot= False, verbose= False, maxparts= 3)

### ============ Simulation Started ============ ###
Time: 0 - [Machine 1] got Part 3 from Queue 1 (capacity= 0)
Time: 0 - [Machine 2] got Part 1 from Queue 2 (capacity= 0)
Time: 4000 - [Conveyor towards Queue 2] GOT Part 3
Time: 4200 - [Conveyor towards Queue 2] PUT Part 3 in the Queue 2
Time: 20000 - [Conveyor towards Queue 4] GOT Part 1
Time: 20000 - [Machine 2] got Part 3 from Queue 2 (capacity= 0)
Time: 20100 - [Conveyor towards Queue 4] PUT Part 1 in the Queue 4
Time: 20101 - [Machine 4] got Part 1 from Queue 4 (capacity= 0)
Time: 23101 - [Conveyor towards Queue 5] GOT Part 1
Time: 23201 - [Conveyor towards Queue 5] PUT Part 1 in the Queue 5
Time: 23202 - [Machine 5] got Part 1 from Queue 5 (capacity= 0)
Time: 25202 - [Terminator] xxx Part 1 terminated xxx
Time: 25202 - [Terminator] Part 4 replaced
Time: 25202 - [Conveyor towards Queue 1] GOT Part 4
Time: 25502 - [Conveyor towards Queue 1] PUT Part 4 in the Queue 1
Time: 25503 - [Machine 1] got Part 4 from Queue 1 (capacity= 0)
T

In [22]:
database_path = "databases/5s_determ/digital_database.db"
testdb = Database(database_path= database_path, event_table= "digital_log", start_time= 0, end_time= 40000)


In [23]:
with sqlite3.connect(database_path) as db:
    db.execute("UPDATE digital_log SET timestamp_real = timestamp;")